In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('./messages.csv')

# Extract the 'Contents' column (your messages)
messages = df['Contents'].tolist()

# Combine all the messages into one large text corpus, ignoring NaN values
full_text = " ".join([str(msg) for msg in messages if pd.notna(msg)])

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
import torch

# Load pre-trained tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is defined
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenize the text
inputs = tokenizer(full_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

# Create a custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize and return as PyTorch tensors
        encoding = self.tokenizer(
            self.texts[idx], 
            return_tensors="pt", 
            padding="max_length", 
            truncation=True, 
            max_length=self.max_length
        )
        return {key: value.squeeze() for key, value in encoding.items()}

# Prepare the dataset
dataset = TextDataset(messages, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2BD1B165-EC09-3F68-BCE4-8FE4E70CA7E2> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <25C510F7-7AEE-3D64-80ED-95874DC6BECD> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
# Prepare the dataset
dataset = TextDataset(messages, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save model checkpoints
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of epochs
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_dir="./logs",  # Directory to save logs
    logging_steps=100,
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start fine-tuning the model
trainer.train()

NameError: name 'TrainingArguments' is not defined